In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from textatistic import Textatistic
import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

import time

ted_df = pd.read_csv("../../data/ted.csv")
tweets_df = pd.read_csv("../../data/russian_tweets.csv")
headlines_df = pd.read_csv("../../data/fakenews.csv")
movie_overviews_df = pd.read_csv("../../data/movie_overviews.csv")
movie_reviews_df = pd.read_csv("../../data/movie_reviews_clean.csv")

## tf-idf vectors for TED talks

### In this exercise, you have been given a corpus ted which contains the transcripts of 500 TED Talks. Your task is to generate the tf-idf vectors for these talks.

### In a later lesson, we will use these vectors to generate recommendations of similar talks based on the transcript.

### Instructions
-    Import TfidfVectorizer from sklearn.
-    Create a TfidfVectorizer object. Name it vectorizer.
-    Generate tfidf_matrix for ted using the fit_transform() method.

In [9]:
ted = ted_df["transcript"]

In [10]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Generate matrix of word vectors
tfidf_matrix = vectorizer.fit_transform(ted)

# Print the shape of tfidf_matrix
print(tfidf_matrix.shape)

(500, 29158)


## Computing dot product

### In this exercise, we will learn to compute the dot product between two vectors, A = (1, 3) and B = (-2, 2), using the numpy library. More specifically, we will use the np.dot() function to compute the dot product of two numpy arrays.

### Instructions
-    Initialize A (1,3) and B (-2,2) as numpy arrays using np.array().
-    Compute the dot product using np.dot() and passing A and B as arguments.

In [11]:
# Initialize numpy vectors
A = np.array([1,3])
B = np.array([-2,2])

# Compute dot product
dot_prod = np.dot(A, B)

# Print dot product
print(dot_prod)

4


## Cosine similarity matrix of a corpus

### In this exercise, you have been given a corpus, which is a list containing five sentences. The corpus is printed in the console. You have to compute the cosine similarity matrix which contains the pairwise cosine similarity score for every pair of sentences (vectorized using tf-idf).

### Remember, the value corresponding to the ith row and jth column of a similarity matrix denotes the similarity score for the ith and jth vector.

### Instructions
-    Initialize an instance of TfidfVectorizer. Name it tfidf_vectorizer.
-    Using fit_transform(), generate the tf-idf vectors for corpus. Name it tfidf_matrix.
-    Use cosine_similarity() and pass tfidf_matrix to compute the cosine similarity matrix cosine_sim.

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

corpus = ['The sun is the largest celestial body in the solar system',
 'The solar system consists of the sun and eight revolving planets',
 'Ra was the Egyptian Sun God',
 'The Pyramids were the pinnacle of Egyptian architecture',
 'The quick brown fox jumps over the lazy dog']

In [14]:
# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the tf-idf vectors for the corpus
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Compute and print the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.36413198 0.18314713 0.18435251 0.16336438]
 [0.36413198 1.         0.15054075 0.21704584 0.11203887]
 [0.18314713 0.15054075 1.         0.21318602 0.07763512]
 [0.18435251 0.21704584 0.21318602 1.         0.12960089]
 [0.16336438 0.11203887 0.07763512 0.12960089 1.        ]]


## Comparing linear_kernel and cosine_similarity

### In this exercise, you have been given tfidf_matrix which contains the tf-idf vectors of a thousand documents. Your task is to generate the cosine similarity matrix for these vectors first using cosine_similarity and then, using linear_kernel.

### We will then compare the computation times for both functions.

### Instructions 1/2
-    Compute the cosine similarity matrix for tfidf_matrix using cosine_similarity.

In [18]:
corpus = movie_overviews_df["overview"].dropna()
tfidf_matrix = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [19]:
# Record start time
start = time.time()

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Print cosine similarity matrix
print(cosine_sim)

# Print time taken
print("Time taken: %s seconds" %(time.time() - start))

[[1.         0.03388343 0.01368106 ... 0.0064274  0.03410257 0.03517135]
 [0.03388343 1.         0.05850441 ... 0.01046158 0.04600773 0.04850257]
 [0.01368106 0.05850441 1.         ... 0.00882085 0.04211172 0.03039092]
 ...
 [0.0064274  0.01046158 0.00882085 ... 1.         0.02631777 0.01108289]
 [0.03410257 0.04600773 0.04211172 ... 0.02631777 1.         0.05593863]
 [0.03517135 0.04850257 0.03039092 ... 0.01108289 0.05593863 1.        ]]
Time taken: 3.1944987773895264 seconds


### Instructions 2/2
-    Compute the cosine similarity matrix for tfidf_matrix using linear_kernel.

In [20]:
from sklearn.metrics.pairwise import linear_kernel

In [21]:
# Record start time
start = time.time()

# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Print cosine similarity matrix
print(cosine_sim)

# Print time taken
print("Time taken: %s seconds" %(time.time() - start))

[[1.         0.03388343 0.01368106 ... 0.0064274  0.03410257 0.03517135]
 [0.03388343 1.         0.05850441 ... 0.01046158 0.04600773 0.04850257]
 [0.01368106 0.05850441 1.         ... 0.00882085 0.04211172 0.03039092]
 ...
 [0.0064274  0.01046158 0.00882085 ... 1.         0.02631777 0.01108289]
 [0.03410257 0.04600773 0.04211172 ... 0.02631777 1.         0.05593863]
 [0.03517135 0.04850257 0.03039092 ... 0.01108289 0.05593863 1.        ]]
Time taken: 3.757575273513794 seconds


## Plot recommendation engine

### In this exercise, we will build a recommendation engine that suggests movies based on similarity of plot lines. You have been given a get_recommendations() function that takes in the title of a movie, a similarity matrix and an indices series as its arguments and outputs a list of most similar movies. indices has already been provided to you.

### You have also been given a movie_plots Series that contains the plot lines of several movies. Your task is to generate a cosine similarity matrix for the tf-idf vectors of these plots.

### Consequently, we will check the potency of our engine by generating recommendations for one of my favorite movies, The Dark Knight Rises.

### Instructions
-    Initialize a TfidfVectorizer with English stop_words. Name it tfidf.
-    Construct tfidf_matrix by fitting and transforming the movie plot data using fit_transform().
-    Generate the cosine similarity matrix cosine_sim using tfidf_matrix. Don't use cosine_similarity()!
-    Use get_recommendations() to generate recommendations for 'The Dark Knight Rises'.

In [26]:
tmp = movie_overviews_df.dropna().set_index("id")

indices = tmp["title"]
movie_plots = tmp["overview"]

def get_recommendations(title, cosine_sim, indices):
    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

## I'm not in the mode to figure out what they did, to get their answer to work

In [ ]:
# Initialize the TfidfVectorizer 
tfidf = TfidfVectorizer(stop_words='english')

# Construct the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(movie_plots)

# Generate the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
 
# Generate recommendations 
print(get_recommendations('The Dark Knight Rises', cosine_sim, indices))

## The recommender function

### In this exercise, we will build a recommender function get_recommendations(), as discussed in the lesson and the previous exercise. As we know, it takes in a title, a cosine similarity matrix, and a movie title and index mapping as arguments and outputs a list of 10 titles most similar to the original title (excluding the title itself).

### You have been given a dataset metadata that consists of the movie titles and overviews. The head of this dataset has been printed to console.

### Instructions
-    Get index of the movie that matches the title by using the title key of indices.
-    Extract the ten most similar movies from sim_scores and store it back in sim_scores.

In [31]:
metadata = movie_overviews_df.dropna()[["title", "tagline"]]

In [32]:
# Generate mapping between titles and index
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

def get_recommendations(title, cosine_sim, indices):
    # Get index of movie that matches title
    idx = indices[title]
    # Sort the movies based on the similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

## TED talk recommender

### In this exercise, we will build a recommendation system that suggests TED Talks based on their transcripts. You have been given a get_recommendations() function that takes in the title of a talk, a similarity matrix and an indices series as its arguments, and outputs a list of most similar talks. indices has already been provided to you.

### You have also been given a transcripts series that contains the transcripts of around 500 TED talks. Your task is to generate a cosine similarity matrix for the tf-idf vectors of the talk transcripts.

### Consequently, we will generate recommendations for a talk titled '5 ways to kill your dreams' by Brazilian entrepreneur Bel Pesce.

### Instructions
-    Initialize a TfidfVectorizer with English stopwords. Name it tfidf.
-    Construct tfidf_matrix by fitting and transforming transcripts.
-    Generate the cosine similarity matrix cosine_sim using tfidf_matrix.
-    Use get_recommendations() to generate recommendations for '5 ways to kill your dreams'.

In [36]:
transcripts = ted_df["transcript"]
## I'm not in the mode to figure out what they did, to get their answer to work

In [ ]:
# Initialize the TfidfVectorizer 
tfidf = TfidfVectorizer(stop_words="english")

# Construct the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(transcripts)

# Generate the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
 
# Generate recommendations 
print(get_recommendations('5 ways to kill your dreams', cosine_sim, indices))

## Generating word vectors

### In this exercise, we will generate the pairwise similarity scores of all the words in a sentence. The sentence is available as sent and has been printed to the console for your convenience.

### Instructions
-    Create a Doc object doc for sent.
-    In the nested loop, compute the similarity between token1 and token2.

In [39]:
sent = 'I like apples and oranges'
nlp = spacy.load("en_core_web_sm")

In [40]:
# Create the doc object
doc = nlp(sent)

# Compute pairwise similarity scores
for token1 in doc:
  for token2 in doc:
    print(token1.text, token2.text, token1.similarity(token2))

I I 1.0
I like -0.06950665265321732
I apples 0.1816101223230362
I and -0.050600726157426834
I oranges -0.01758648455142975
like I -0.06950665265321732
like like 1.0
like apples 0.10200604051351547
like and 0.0523926205933094
like oranges 0.08973564207553864
apples I 0.1816101223230362
apples like 0.10200604051351547
apples apples 1.0
apples and -0.0011530559277161956
apples oranges 0.41077685356140137
and I -0.050600726157426834
and like 0.0523926205933094
and apples -0.0011530559277161956
and and 1.0
and oranges 0.028571248054504395
oranges I -0.01758648455142975
oranges like 0.08973564207553864
oranges apples 0.41077685356140137
oranges and 0.028571248054504395
oranges oranges 1.0


/tmp/ipykernel_380283/2929505010.py:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(token1.text, token2.text, token1.similarity(token2))


## Computing similarity of Pink Floyd songs

### In this final exercise, you have been given lyrics of three songs by the British band Pink Floyd, namely 'High Hopes', 'Hey You' and 'Mother'. The lyrics to these songs are available as hopes, hey and mother respectively.

### Your task is to compute the pairwise similarity between mother and hopes, and mother and hey.

### Instructions
-    Create Doc objects for mother, hopes and hey.
-    Compute the similarity between mother and hopes.
-    Compute the similarity between mother and hey.

In [41]:
mother = "\nMother do you think they'll drop the bomb?\nMother do you think they'll like this song?\nMother do you think they'll try to break my balls?\nOoh, ah\nMother should I build the wall?\nMother should I run for President?\nMother should I trust the government?\nMother will they put me in the firing mine?\nOoh ah,\nIs it just a waste of time?\nHush now baby, baby, don't you cry.\nMama's gonna make all your nightmares come true.\nMama's gonna put all her fears into you.\nMama's gonna keep you right here under her wing.\nShe won't let you fly, but she might let you sing.\nMama's gonna keep baby cozy and warm.\nOoh baby, ooh baby, ooh baby,\nOf course mama's gonna help build the wall.\nMother do you think she's good enough, for me?\nMother do you think she's dangerous, to me?\nMother will she tear your little boy apart?\nOoh ah,\nMother will she break my heart?\nHush now baby, baby don't you cry.\nMama's gonna check out all your girlfriends for you.\nMama won't let anyone dirty get through.\nMama's gonna wait up until you get in.\nMama will always find out where you've been.\nMama's gonna keep baby healthy and clean.\nOoh baby, ooh baby, ooh baby,\nYou'll always be baby to me.\nMother, did it need to be so high?\n"
hopes = "\nBeyond the horizon of the place we lived when we were young\nIn a world of magnets and miracles\nOur thoughts strayed constantly and without boundary\nThe ringing of the division bell had begun\nAlong the Long Road and on down the Causeway\nDo they still meet there by the Cut\nThere was a ragged band that followed in our footsteps\nRunning before times took our dreams away\nLeaving the myriad small creatures trying to tie us to the ground\nTo a life consumed by slow decay\nThe grass was greener\nThe light was brighter\nWhen friends surrounded\nThe nights of wonder\nLooking beyond the embers of bridges glowing behind us\nTo a glimpse of how green it was on the other side\nSteps taken forwards but sleepwalking back again\nDragged by the force of some in a tide\nAt a higher altitude with flag unfurled\nWe reached the dizzy heights of that dreamed of world\nEncumbered forever by desire and ambition\nThere's a hunger still unsatisfied\nOur weary eyes still stray to the horizon\nThough down this road we've been so many times\nThe grass was greener\nThe light was brighter\nThe taste was sweeter\nThe nights of wonder\nWith friends surrounded\nThe dawn mist glowing\nThe water flowing\nThe endless river\nForever and ever\n"
hey = "\nHey you, out there in the cold\nGetting lonely, getting old\nCan you feel me?\nHey you, standing in the aisles\nWith itchy feet and fading smiles\nCan you feel me?\nHey you, don't help them to bury the light\nDon't give in without a fight\nHey you out there on your own\nSitting naked by the phone\nWould you touch me?\nHey you with you ear against the wall\nWaiting for someone to call out\nWould you touch me?\nHey you, would you help me to carry the stone?\nOpen your heart, I'm coming home\nBut it was only fantasy\nThe wall was too high\nAs you can see\nNo matter how he tried\nHe could not break free\nAnd the worms ate into his brain\nHey you, out there on the road\nAlways doing what you're told\nCan you help me?\nHey you, out there beyond the wall\nBreaking bottles in the hall\nCan you help me?\nHey you, don't tell me there's no hope at all\nTogether we stand, divided we fall\n"

In [42]:
# Create Doc objects
mother_doc = nlp(mother)
hopes_doc = nlp(hopes)
hey_doc = nlp(hey)

# Print similarity between mother and hopes
print(mother_doc.similarity(hopes_doc))

# Print similarity between mother and hey
print(mother_doc.similarity(hey_doc))

0.4734780725243333
0.817943786306124


/tmp/ipykernel_380283/1287196326.py:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(mother_doc.similarity(hopes_doc))
/tmp/ipykernel_380283/1287196326.py:10: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of t